## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,66.47,63,94,2.77,overcast clouds
1,1,Pisco,PE,-13.7000,-76.2167,66.25,77,0,8.05,clear sky
2,2,Comodoro Rivadavia,AR,-45.8667,-67.5000,35.46,64,0,4.61,clear sky
3,3,Los Llanos De Aridane,ES,28.6585,-17.9182,60.55,92,75,11.50,broken clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,61.32,71,51,3.00,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,66.47,63,94,2.77,overcast clouds
1,1,Pisco,PE,-13.7000,-76.2167,66.25,77,0,8.05,clear sky
3,3,Los Llanos De Aridane,ES,28.6585,-17.9182,60.55,92,75,11.50,broken clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,61.32,71,51,3.00,broken clouds
5,5,Poum,NC,-20.2333,164.0167,77.56,70,41,9.53,scattered clouds
6,6,Grand-Santi,GF,4.2500,-54.3833,73.42,98,94,3.24,overcast clouds
7,7,Busselton,AU,-33.6500,115.3333,64.99,57,0,2.98,clear sky
8,8,Saint-Philippe,RE,-21.3585,55.7679,74.79,83,44,14.74,scattered clouds
9,9,Bluff,NZ,-46.6000,168.3333,66.79,72,98,5.17,overcast clouds
10,10,Kahului,US,20.8947,-156.4700,80.62,66,40,25.32,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
np.where(pd.isnull(preferred_cities_df))

(array([ 11,  57, 239, 330, 436, 448], dtype=int64),
 array([2, 2, 2, 2, 2, 2], dtype=int64))

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace('', np.nan, inplace=True)
clean_df = preferred_cities_df.dropna()

C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,66.47,overcast clouds,-46.1927,168.8643,
1,Pisco,PE,66.25,clear sky,-13.7000,-76.2167,
3,Los Llanos De Aridane,ES,60.55,broken clouds,28.6585,-17.9182,
4,Port Elizabeth,ZA,61.32,broken clouds,-33.9180,25.5701,
5,Poum,NC,77.56,scattered clouds,-20.2333,164.0167,
6,Grand-Santi,GF,73.42,overcast clouds,4.2500,-54.3833,
7,Busselton,AU,64.99,clear sky,-33.6500,115.3333,
8,Saint-Philippe,RE,74.79,scattered clouds,-21.3585,55.7679,
9,Bluff,NZ,66.79,overcast clouds,-46.6000,168.3333,
10,Kahului,US,80.62,scattered clouds,20.8947,-156.4700,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,66.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
1,Pisco,PE,66.25,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
3,Los Llanos De Aridane,ES,60.55,broken clouds,28.6585,-17.9182,Valle Aridane
4,Port Elizabeth,ZA,61.32,broken clouds,-33.9180,25.5701,39 On Nile Guest House
6,Grand-Santi,GF,73.42,overcast clouds,4.2500,-54.3833,Hotel La Villa Couacou
7,Busselton,AU,64.99,clear sky,-33.6500,115.3333,Observatory Guest House
8,Saint-Philippe,RE,74.79,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
9,Bluff,NZ,66.79,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
10,Kahului,US,80.62,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
11,Dwarka,IN,85.37,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))